In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=77af92d734f84ff580fc67ac6211e137cc2c604b3c17e7f288ee2cc2809e434a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

22/01/09 21:26:49 WARN Utils: Your hostname, daniyal resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
22/01/09 21:26:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/09 21:26:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/09 21:26:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/09 21:26:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


ModuleNotFoundError: No module named 'org'

In [5]:

df=spark.read.csv('cancerdata_test.csv',header=True,inferSchema=True)
df=df.drop('_c32').drop('id')
df.printSchema()

train,test=df.randomSplit([0.7,0.3])

root
 |-- diagnosis: string (nullable = true)
 |-- radius_mean: double (nullable = true)
 |-- texture_mean: double (nullable = true)
 |-- perimeter_mean: double (nullable = true)
 |-- area_mean: double (nullable = true)
 |-- smoothness_mean: double (nullable = true)
 |-- compactness_mean: double (nullable = true)
 |-- concavity_mean: double (nullable = true)
 |-- concave points_mean: double (nullable = true)
 |-- symmetry_mean: double (nullable = true)
 |-- fractal_dimension_mean: double (nullable = true)
 |-- radius_se: double (nullable = true)
 |-- texture_se: double (nullable = true)
 |-- perimeter_se: double (nullable = true)
 |-- area_se: double (nullable = true)
 |-- smoothness_se: double (nullable = true)
 |-- compactness_se: double (nullable = true)
 |-- concavity_se: double (nullable = true)
 |-- concave points_se: double (nullable = true)
 |-- symmetry_se: double (nullable = true)
 |-- fractal_dimension_se: double (nullable = true)
 |-- radius_worst: double (nullable = true)


In [130]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.ml import Pipeline




In [143]:
from pyspark.ml.feature import VectorAssembler
cols =[c for c in df.columns if c!='diagnosis']

va=VectorAssembler(inputCols=cols,outputCol='vafeatures')


In [5]:
#scaling
from pyspark.ml.feature import StandardScaler

scaler=StandardScaler(inputCol='vafeatures',outputCol='scaledFeatures')

#encoding
from pyspark.ml.feature import StringIndexer
sInd=StringIndexer(inputCol='diagnosis',outputCol='label')







AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [4]:
#Model
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(
featuresCol='scaledFeatures',
    predictionCol='prediction',
    probabilityCol='probability',
    rawPredictionCol='rawPrediction',
    maxDepth=5,
    maxBins=32,
    minInstancesPerNode=1,
    minInfoGain=0.0,
    maxMemoryInMB=256,
    cacheNodeIds=False,
    checkpointInterval=1,
    impurity='gini',
    numTrees=10,
    featureSubsetStrategy='auto',
    seed=None,
    subsamplingRate=0.6)



AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [147]:
#Step 4 Evaulator
#-----------------------------------------------------Evaulator of model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol=rf.getLabelCol(), predictionCol=rf.getPredictionCol(), metricName="accuracy")




In [148]:
print(rf.explainParams()+"\n")

bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False, current: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10, current: 1)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onet

In [149]:

param= (ParamGridBuilder()
               .addGrid(rf.maxDepth, [2, 5])
               .addGrid(rf.maxBins, [5, 10])
               .addGrid(rf.numTrees, [5, 20])
             .build())


In [150]:
cv=CrossValidator(estimator=rf,evaluator=evaluator,numFolds=2,parallelism=8,estimatorParamMaps=param,seed=12)

In [151]:
from pyspark.ml.pipeline import Pipeline
pipe=Pipeline(stages=[va,scaler,sInd,cv])



In [152]:
pModel=pipe.fit(train)

In [158]:
bestModel=pModel.stages[-1].bestModel.explainParams
bestModel

<bound method Params.explainParams of RandomForestClassificationModel: uid=RandomForestClassifier_1caad440457a, numTrees=20, numClasses=4, numFeatures=30>

In [154]:
evalResult=pModel.transform(test)
evalResult.show()


+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symme

In [156]:
result=evaluator.setMetricName('f1').evaluate(evalResult)
result


0.8151751414997841

In [157]:
result=evaluator.setMetricName('accuracy').evaluate(evalResult)
result


0.8648648648648649